# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770282314248                   -0.53    9.0         
  2   -2.771676309939       -2.86       -1.30    1.0    213ms
  3   -2.771713529678       -4.43       -2.72    1.0    240ms
  4   -2.771714700722       -5.93       -3.50    2.0    282ms
  5   -2.771714712538       -7.93       -3.84    2.0    221ms
  6   -2.771714715222       -8.57       -5.41    1.0    211ms
  7   -2.771714715249      -10.57       -5.65    2.0    267ms
  8   -2.771714715250      -12.42       -6.11    2.0    230ms
  9   -2.771714715250      -13.83       -6.80    1.0    243ms
 10   -2.771714715250      -14.88       -7.19    1.0    233ms
 11   -2.771714715250   +    -Inf       -8.26    2.0    238ms


-0.0001345742335706158

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770469208229                   -0.53    8.0         
  2   -2.771756801041       -2.89       -1.30    1.0    257ms
  3   -2.771800350568       -4.36       -2.71    1.0    193ms
  4   -2.771802059070       -5.77       -3.49    2.0    310ms
  5   -2.771802073442       -7.84       -4.03    2.0    234ms
  6   -2.771802074459       -8.99       -5.03    1.0    203ms
  7   -2.771802074476      -10.76       -6.11    2.0    272ms
  8   -2.771802074476      -13.61       -6.26    1.0    213ms
  9   -2.771802074476      -13.75       -7.27    1.0    242ms
 10   -2.771802074476      -14.05       -7.32    2.0    278ms
 11   -2.771802074476   +  -13.94       -7.76    1.0    220ms
 12   -2.771802074476      -13.91       -7.85    1.0    247ms
 13   -2.771802074476   +  -13.94       -8.35    1.0    233ms


0.01761222097493452

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.0001345742335706158
Displaced dipole:  0.01761222097493452
Polarizability :   1.7746795208505135


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920944864e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766553477908e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852761460409e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694608678150e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088784047525e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.278267592691e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 8.402028967088e-13
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 8.402028967088e-13
[ Info: GMRES linsolve in iter 2; step 1: normres = 8.894289963241e-10
[ Info: GMRES linsolve in iter 2; step 2: normres = 4.840917199025e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 1.139234678521e-12
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 1.139234678521e-12
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 1.1392349150419438e-12
└ *  number of opera

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.